In [6]:
import sys
sys.path.append('../')
import simulation_tools as sim

import pandas as pd
import numpy as np

from cmdstanpy import CmdStanModel
from IPython.display import display

In [7]:
# Remove cmdstanpy logs
import logging
logger = logging.getLogger('cmdstanpy')
logger.addHandler(logging.NullHandler())
logger.propagate = False
logger.setLevel(logging.CRITICAL)

In [8]:
num_repetitions = 2
model = CmdStanModel(stan_file="../basic_mlm.stan")

# Balanced design has 50% of subjects in treatment
# Varying design has 50% of subjects in treatment for the
# first 5 experiments and then increases linearly to 95%
prop_const = np.full(38, 0.5)
prop_varying = np.concatenate(
    (np.full(5, 0.5), np.linspace(0.5, 0.95, sim.CHICK_NUM_EXPTS - 5))
)
params = {
    "num_subjects_per_expt": [np.full(sim.CHICK_NUM_EXPTS, sim.CHICK_NUM_SUBJECTS_PER_EXPT)],
    "prop_treatment": [prop_const, prop_varying],
    "mu_b": [0],
    "mu_theta": [sim.CHICK_MU_THETA],
    "sigma_b": sim.CHICK_SIGMA_B_GRID,
    "sigma_theta": [sim.CHICK_SIGMA_THETA],
    "sigma_treatment": [sim.CHICK_SIGMA_TREATMENT],
    "sigma_control": [sim.CHICK_SIGMA_CONTROL],
}

In [9]:
df = sim.evaluate_params_means(model, num_repetitions, params)

Parameter Combination:   0%|          | 0/22 [00:00<?, ?it/s]

Repetition:   0%|          | 0/2 [00:00<?, ?it/s]

Repetition:   0%|          | 0/2 [00:00<?, ?it/s]

Repetition:   0%|          | 0/2 [00:00<?, ?it/s]

Repetition:   0%|          | 0/2 [00:00<?, ?it/s]

...

Repetition:   0%|          | 0/2 [00:00<?, ?it/s]

Repetition:   0%|          | 0/2 [00:00<?, ?it/s]

Repetition:   0%|          | 0/2 [00:00<?, ?it/s]

Repetition:   0%|          | 0/2 [00:00<?, ?it/s]

Repetition:   0%|          | 0/2 [00:00<?, ?it/s]

In [10]:
from datetime import datetime

# Get the current timestamp
timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

# Save df to a CSV file
df.to_csv(f"fake_data/df_{timestamp}.csv", index=False)

In [11]:
# Filter rows where "prop_treatment" matches const_prop
# df_const_p = df[df["prop_treatment"].apply(lambda x: np.array_equal(x, prop_const))]
# df_vary_p = df[df["prop_treatment"].apply(lambda x: np.array_equal(x, prop_varying))]

In [12]:
import plotly.express as px

# Create the plot using Plotly
fig_mse = px.line(
    df.assign(LineStyle=df["prop_treatment"].apply(
        lambda x: "Constant p" if np.array_equal(x, prop_const) else "Varying p"
    )),
    x="sigma_b",
    y="sample_mse",
    color="estimator",
    line_dash="LineStyle",
    markers=True,
    title="MSE for Different Estimators vs. sigma_b",
    labels={"sigma_b": "sigma_b", "sample_mse": "Sample MSE"},
)

# Update the legend to remove linestyle
fig_mse.update_layout(legend_title_text="Estimator")
fig_mse.show()

In [13]:
# Create the plot using Plotly
fig_type_s = px.line(
    df.assign(LineStyle=df["prop_treatment"].apply(
        lambda x: "Constant p" if np.array_equal(x, prop_const) else "Varying p"
    )),
    x="sigma_b",
    y="type_s_rate",
    color="estimator",
    line_dash="LineStyle",
    markers=True,
    title="Type S Error for Different Estimators vs. sigma_b",
    labels={"sigma_b": "sigma_b", "type_s_rate": "Type S Error"},
)

# Update the legend to remove linestyle
fig_type_s.update_layout(legend_title_text="Estimator")
fig_type_s.show()


In [14]:
# Create the plot using Plotly
fig_prop_signif = px.line(
    df.assign(LineStyle=df["prop_treatment"].apply(
        lambda x: "Constant p" if np.array_equal(x, prop_const) else "Varying p"
    )),
    x="sigma_b",
    y="prop_signif",
    color="estimator",
    line_dash="LineStyle",
    markers=True,
    title="Proportion of Significant Results vs. sigma_b",
    labels={"sigma_b": "sigma_b", "prop_signif": "Proportion of Significant Results"},
)

# Update the legend to remove linestyle
fig_prop_signif.update_layout(legend_title_text="Estimator")
fig_prop_signif.show()

In [15]:
# Create the plot using Plotly
fig_rank_corr = px.line(
    df.assign(LineStyle=df["prop_treatment"].apply(
        lambda x: "Constant p" if np.array_equal(x, prop_const) else "Varying p"
    )),
    x="sigma_b",
    y="rank_corr",
    color="estimator",
    line_dash="LineStyle",
    markers=True,
    title="Rank Correlation for Different Estimators vs. sigma_b",
    labels={"sigma_b": "sigma_b", "rank_corr": "Rank Correlation"},
)

# Update the legend to remove linestyle
fig_rank_corr.update_layout(legend_title_text="Estimator")
fig_rank_corr.show()